## Food2Fork API ##
www.food2fork.com/about/api

In [1]:
import requests
import urllib2
import json
from bs4 import BeautifulSoup

#### Load API credentials ####

In [2]:
def loadCredentials():
    filename = 'secrets.txt'
    for line in open(filename).readlines():
        if "API" in line:    
            api_key = line.split(": ")[1].translate(None,'\n')
        
    return api_key

In [3]:
API_KEY = loadCredentials()

In [4]:
URL_API = 'http://food2fork.com/api'
URL_SEARCH = URL_API + '/search/?'
URL_GET = URL_API + '/get/?'

# Base URLs
search_url = "http://food2fork.com/api/search"
recipe_url = "http://food2fork.com/api/get"

#### Make a GET request to the API ####

In [6]:
# api_request = search_url + ""
page = 1
while True:    
    api_request = "http://food2fork.com/api/search?key="+API_KEY+"&q=shredded%20chicken&page="+str(page)

    # I don't understand headers well enough yet, but I got a 403 error without this header
    # https://en.wikipedia.org/wiki/User_agent
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}

    request = urllib2.Request(api_request, headers=hdr)
    response = urllib2.urlopen(request)
    raw = response.read()
    json_obj = json.loads(raw)    
    
    if json_obj['count']!=30 or page == 2:
        print("\nPage {p} is the last page.".format(p=page))
        break
    elif page==1:
        print("\nPage: {p}, ".format(p=page)),
    else:
        print("{p}, ".format(p=page)),
        
    page += 1
    
#     json_obj.viewkeys()



Page: 1,  
Page 2 is the last page.


In [10]:
[(recipe['title'],recipe['source_url']) for recipe in json_obj['recipes'] if "allrecipes" in recipe['source_url']]

[(u'Simple, Classic Chicken Potpie',
  u'http://allrecipes.com/Recipe/Simple-Classic-Chicken-Potpie/Detail.aspx'),
 (u'Sour Cream Chicken Enchiladas',
  u'http://allrecipes.com/Recipe/Sour-Cream-Chicken-Enchiladas/Detail.aspx'),
 (u'Honey-Lime Enchiladas',
  u'http://allrecipes.com/Recipe/Honey-Lime-Enchiladas/Detail.aspx')]

### Scrape ingredients from an AllRecipes.com page ###

In [53]:
def getIngredients(URL):
    page = requests.get(URL)
    html = BeautifulSoup(page.text, "html.parser")
    
    ingredients = [result.get_text().encode('ascii','ignore').lower().strip("\n") for result in html.find_all("li", class_="checkList__line")]
    ingredients = [ingredient.strip('advertisement').strip('\n') for ingredient in ingredients]
    return ingredients

In [58]:
[(recipe['title'], getIngredients(recipe['source_url'])) for recipe in json_obj['recipes'] if "allrecipes" in recipe['source_url']]

[(u'Simple, Classic Chicken Potpie',
  ['6 cups roasted, shredded rotisserie chicken',
   '1 (15 ounce) package refrigerated pie crusts',
   '2 tablespoons vegetable oil',
   '2 medium onions, chopped',
   '3 small stalks celery, cut crosswise 1/4-inch thick',
   '2 cups chicken broth',
   '1 (12 fluid ounce) can evaporated milk',
   '1/3 cup butter',
   '9 tablespoons all-purpose flour',
   '3/4 teaspoon dried thyme',
   '1/4 cup dry sherry or white wine',
   'lt and ground pepper',
   '1/4 cup chopped fresh parsley',
   '1 (16 ounce) package frozen peas and carrots, not thawed',
   ' all ingredients to l',
   '',
   ' all ingredients to l']),
 (u'Sour Cream Chicken Enchiladas',
  ['1 bunch cilantro',
   '1 cup sour cream',
   '2 (7 ounce) cans jalapeno salsa',
   '2 (7 ounce) cans prepared green chile salsa',
   '2 skinless, boneless chicken breast halves, cooked and shredded',
   '1 onion',
   '12 (6 inch) flour tortillas',
   '2 cups shredded cheddar cheese',
   ' all ingredients t